In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats





df = pd.read_csv("..\\data\\benin-malanville.csv")
#print(df.head(10))
#print(df.describe())



In [ ]:
print("Description of the numerical columns:")
print(df.describe())
print("Description of the categorical columns:")
print(df.describe(include=['object']))
print("Description of all columns:")
print(df.describe(include='all'))
print("An overview of the dataset:")
print(df.head())
print("The number of rows and columns in the dataset:")
print(df.shape)
print("An overview of the dataset:")
print(df.tail())
print("An overview of the dataset: 5 random rows")
print(df.sample(5))
print("An overview of the dataset: the shape of the dataset")
print(df.info())
print("Check for missing values:")
print(df.isna().sum())
print("Check for duplicates:")
print(df.duplicated().sum())
